# Chapter 2: End-to-End Machine Learning Project

Pada chapter ini, kita akan mempelajari bagaimana sebuah proyek Machine Learning dikerjakan **secara menyeluruh dari awal hingga akhir** (*end-to-end*).

Berbeda dengan chapter sebelumnya yang berfokus pada konsep dasar Machine Learning, chapter ini menempatkan kita pada **skenario dunia nyata**, seolah-olah kita adalah seorang data scientist yang baru direkrut oleh sebuah perusahaan.

Tujuan utama dari chapter ini adalah memahami **alur kerja sistematis** dalam proyek Machine Learning, mulai dari memahami masalah bisnis, mengumpulkan data, melakukan eksplorasi data, mempersiapkan data, memilih model, hingga mengevaluasi dan mempersiapkan sistem untuk digunakan secara nyata.

Pendekatan end-to-end ini sangat penting karena dalam praktik profesional, tantangan terbesar Machine Learning sering kali bukan pada algoritma, melainkan pada **data, asumsi, dan proses pengambilan keputusan**.

## 1. Look at the Big Picture

Langkah pertama dalam proyek Machine Learning adalah memahami gambaran besar dari masalah yang ingin diselesaikan.

Pada chapter ini, kita berperan sebagai data scientist di sebuah perusahaan real estate yang ingin membangun sistem prediksi **median harga rumah** di California berdasarkan data sensus.

Model yang dibangun nantinya akan digunakan sebagai bagian dari sistem yang lebih besar untuk membantu pengambilan keputusan investasi.

### 1.1 Framing the Problem

Sebelum menulis satu baris kode pun, sangat penting untuk memahami **apa sebenarnya masalah yang ingin diselesaikan**.

Beberapa pertanyaan kunci yang perlu dijawab:
- Apakah ini masalah *supervised* atau *unsupervised learning*?
- Apakah ini tugas *classification* atau *regression*?
- Apakah sistem perlu belajar secara *batch* atau *online*?

Dalam kasus ini:
- Data memiliki label (harga rumah) → **Supervised Learning**
- Target berupa nilai kontinu → **Regression**
- Menggunakan banyak fitur untuk memprediksi satu nilai → **Multiple Regression**
- Data relatif statis dan tidak terlalu besar → **Batch Learning**

### 1.2 Select a Performance Measure

Setelah masalah diformulasikan dengan jelas, langkah berikutnya adalah menentukan **metrik evaluasi**.

Untuk masalah regresi, salah satu metrik yang paling umum digunakan adalah **Root Mean Square Error (RMSE)**.

RMSE memberikan gambaran seberapa besar kesalahan prediksi model secara rata-rata, dengan memberikan penalti lebih besar pada error yang besar.

Secara intuitif:
- RMSE kecil → prediksi model mendekati nilai sebenarnya
- RMSE besar → model sering membuat kesalahan besar

Selain RMSE, terkadang digunakan juga **Mean Absolute Error (MAE)**.

Perbedaan utama:
- RMSE lebih sensitif terhadap *outlier*
- MAE lebih robust terhadap *outlier*

Pemilihan metrik harus disesuaikan dengan konteks bisnis dan karakteristik data.

## 2. Get the Data

Setelah memahami gambaran besar dan tujuan proyek, langkah berikutnya adalah **mengumpulkan dan memuat data**.

Pada proyek ini, kita menggunakan dataset **California Housing** yang berisi informasi sensus mengenai perumahan di California.

Dataset ini mencakup berbagai fitur seperti lokasi geografis, jumlah kamar, jumlah penduduk, dan median pendapatan, yang semuanya berpotensi memengaruhi harga rumah.

### 2.1 Create the Workspace

Sebelum memuat data, kita perlu menyiapkan *workspace* yang rapi.

Langkah ini mencakup:
- mengimpor library yang dibutuhkan,
- menyiapkan struktur folder untuk menyimpan data,
- memastikan eksperimen dapat direproduksi.

Pendekatan ini sangat penting dalam proyek Machine Learning nyata agar proses kerja terorganisir dan dapat diulang.

In [ ]:
import os
import tarfile
import urllib.request


Kode di atas mengimpor modul standar Python yang digunakan untuk:
- mengelola direktori,
- mengekstrak file terkompresi,
- serta mengunduh data dari internet.

### 2.2 Downloading the Data

Dataset California Housing disediakan dalam bentuk file terkompresi. Kita akan membuat fungsi untuk:
- mengunduh dataset jika belum tersedia,
- mengekstraknya ke folder lokal.

Pendekatan ini memastikan bahwa notebook dapat dijalankan ulang tanpa harus mengunduh data berulang kali.

In [ ]:
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

In [ ]:
def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

Fungsi `fetch_housing_data()` bertanggung jawab untuk mengunduh dan mengekstrak dataset.

Dengan membungkus proses ini ke dalam fungsi, kita menjaga kode tetap bersih dan mudah digunakan kembali.

In [ ]:
fetch_housing_data()

Setelah fungsi dijalankan, dataset akan tersedia secara lokal di direktori `datasets/housing`.

### 2.3 Loading the Data

Setelah data tersedia secara lokal, langkah berikutnya adalah memuat data ke dalam struktur yang mudah dianalisis.

Pada proyek ini, kita menggunakan **pandas DataFrame**, yang menyediakan berbagai fungsi untuk eksplorasi dan manipulasi data.

In [ ]:
import pandas as pd


In [ ]:
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [ ]:
housing = load_housing_data()
housing.head()

DataFrame di atas menampilkan lima baris pertama dari dataset California Housing.

Setiap baris merepresentasikan satu distrik sensus, dan setiap kolom merepresentasikan fitur atau target yang akan digunakan dalam proses Machine Learning.

## 3. Take a Quick Look at the Data Structure

Setelah data dimuat, langkah penting berikutnya adalah **memahami struktur data secara umum**.

Pada tahap ini, tujuan kita bukan melakukan analisis mendalam, melainkan:
- mengetahui jumlah baris dan kolom,
- memahami tipe data setiap fitur,
- mengidentifikasi adanya nilai yang hilang (*missing values*),
- serta mengenali fitur kategorikal dan numerik.

Langkah ini membantu kita merencanakan proses pembersihan dan persiapan data pada tahap selanjutnya.

### 3.1 Data Overview with `info()`

Fungsi `info()` pada pandas memberikan ringkasan singkat mengenai DataFrame, termasuk jumlah non-null value dan tipe data setiap kolom.

In [ ]:
housing.info()

Dari output `info()`, kita dapat mengamati bahwa sebagian besar fitur bertipe numerik.

Namun, terdapat satu fitur bertipe objek, yaitu `ocean_proximity`, yang menunjukkan bahwa fitur tersebut bersifat **kategorikal**.

Selain itu, terlihat bahwa kolom `total_bedrooms` memiliki jumlah non-null value yang lebih sedikit, yang mengindikasikan adanya **nilai yang hilang**.

### 3.2 Descriptive Statistics

Untuk mendapatkan gambaran statistik dasar dari fitur numerik, kita dapat menggunakan metode `describe()`.

Metode ini menampilkan informasi seperti nilai minimum, maksimum, rata-rata, dan standar deviasi.

In [ ]:
housing.describe()

Statistik deskriptif ini memberikan banyak informasi penting, misalnya:
- rentang nilai setiap fitur,
- adanya fitur dengan skala yang sangat berbeda,
- serta kemungkinan keberadaan outlier.

Perbedaan skala ini menjadi alasan mengapa **feature scaling** sering diperlukan sebelum melatih model.

### 3.3 Categorical Attribute: `ocean_proximity`

Kolom `ocean_proximity` merupakan satu-satunya fitur kategorikal dalam dataset ini.

Untuk memahami distribusi kategorinya, kita dapat menghitung jumlah kemunculan setiap kategori.

In [ ]:
housing["ocean_proximity"].value_counts()

Distribusi kategori ini menunjukkan bagaimana distrik sensus tersebar berdasarkan kedekatannya dengan laut.

Informasi ini penting karena lokasi geografis memiliki pengaruh besar terhadap harga rumah, sehingga fitur ini kemungkinan besar memiliki kontribusi signifikan terhadap prediksi.

## 4. Create a Test Set

Sebelum melakukan eksplorasi data lebih lanjut atau melatih model, sangat penting untuk **memisahkan test set sejak awal**.

Tujuan utama dari test set adalah untuk memberikan estimasi performa model pada data yang benar-benar belum pernah dilihat selama proses training.

Jika test set digunakan terlalu dini atau terpapar selama eksplorasi, maka evaluasi akhir model dapat menjadi bias.

### 4.1 Random Sampling

Pendekatan paling sederhana untuk membagi data adalah **random sampling**, yaitu memilih sebagian data secara acak sebagai test set.

Namun, pendekatan ini memiliki kelemahan, terutama ketika dataset relatif kecil atau memiliki distribusi fitur yang tidak merata.

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)
len(train_set), len(test_set)

Pembagian ini menghasilkan sekitar 80% data untuk training dan 20% data untuk testing.

Penggunaan `random_state` memastikan bahwa pembagian data bersifat **reproducible**.

### 4.2 Stratified Sampling

Random sampling dapat menghasilkan test set yang tidak merepresentasikan populasi dengan baik.

Untuk mengatasi hal ini, digunakan **stratified sampling**, yaitu memastikan bahwa distribusi fitur penting tetap terjaga pada training dan test set.

Dalam proyek ini, fitur `median_income` dianggap sangat penting untuk prediksi harga rumah.

Oleh karena itu, kita membuat kategori pendapatan dan menggunakan kategori ini sebagai dasar stratifikasi.

In [ ]:
housing["income_cat"] = pd.cut(
    housing["median_income"],
    bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
    labels=[1, 2, 3, 4, 5]
)

Fitur `income_cat` mengelompokkan nilai `median_income` ke dalam beberapa kategori diskrit.

Pendekatan ini membantu memastikan bahwa distribusi pendapatan tetap konsisten antara training dan test set.

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

Dengan stratified sampling, training dan test set memiliki distribusi `income_cat` yang sangat mirip dengan dataset asli.

Pendekatan ini menghasilkan evaluasi model yang lebih adil dan representatif.

Setelah pembagian data selesai, fitur `income_cat` tidak lagi dibutuhkan dan dapat dihapus untuk mencegah *data leakage*.

In [ ]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

## 5. Discover and Visualize the Data to Gain Insights

Setelah training dan test set dipisahkan dengan benar, kita dapat mulai melakukan **eksplorasi data secara lebih mendalam**.

Tahap ini bertujuan untuk:
- memahami pola dan distribusi data,
- mengidentifikasi hubungan antar fitur,
- menemukan anomali atau outlier,
- serta mendapatkan intuisi awal tentang faktor yang memengaruhi harga rumah.

Seluruh eksplorasi dilakukan **hanya pada training set**, untuk menghindari *data leakage*.

Untuk mempermudah eksplorasi, kita membuat salinan dari training set.

In [ ]:
housing = strat_train_set.copy()

### 5.1 Visualizing Geographical Data

Karena dataset California Housing memiliki informasi geografis (longitude dan latitude), kita dapat memvisualisasikan lokasi distrik sensus untuk melihat pola spasial.

Visualisasi ini sering kali memberikan wawasan yang sulit diperoleh hanya dari tabel angka.

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)

Plot di atas menunjukkan kepadatan distrik sensus di California. Area dengan titik yang lebih padat merepresentasikan wilayah dengan populasi lebih tinggi.

### 5.2 Visualizing Housing Prices

Untuk mendapatkan insight yang lebih kaya, kita dapat memvisualisasikan harga rumah dengan:
- ukuran titik merepresentasikan populasi,
- warna titik merepresentasikan median harga rumah.

In [ ]:
housing.plot(
    kind="scatter",
    x="longitude",
    y="latitude",
    alpha=0.4,
    s=housing["population"] / 100,
    label="population",
    figsize=(10, 7),
    c="median_house_value",
    cmap=plt.get_cmap("jet"),
    colorbar=True,
)
plt.legend()

Visualisasi ini memperlihatkan bahwa harga rumah cenderung lebih tinggi di wilayah pesisir dan area metropolitan besar.

Insight ini menegaskan bahwa **lokasi geografis merupakan faktor penting** dalam prediksi harga rumah.

### 5.3 Looking for Correlations

Setelah visualisasi, langkah selanjutnya adalah mengukur hubungan antar fitur secara kuantitatif menggunakan **koefisien korelasi**.

Korelasi membantu kita memahami fitur mana yang memiliki hubungan paling kuat dengan target.

In [ ]:
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

Hasil korelasi menunjukkan bahwa `median_income` memiliki korelasi paling kuat dengan `median_house_value`.

Informasi ini sangat berguna dalam proses feature engineering dan pemilihan model.

### 5.4 Scatter Matrix

Untuk memvisualisasikan hubungan antar beberapa fitur penting sekaligus, kita dapat menggunakan **scatter matrix**.

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms",
              "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12, 8))

Scatter matrix memperjelas hubungan linear antara `median_income` dan `median_house_value`, serta membantu mengidentifikasi pola non-linear dan outlier.

## 6. Experimenting with Attribute Combinations

Pada tahap ini, kita mulai melakukan **feature engineering**, yaitu menciptakan fitur baru dari fitur yang sudah ada.

Tujuan utama feature engineering adalah mengekspresikan informasi dalam data dengan cara yang lebih bermakna bagi model Machine Learning.

Sering kali, fitur hasil kombinasi justru memiliki korelasi yang lebih kuat terhadap target dibandingkan fitur mentahnya.

### 6.1 Creating New Attributes

Dalam dataset perumahan ini, beberapa fitur mentah kurang informatif jika digunakan secara terpisah.

Sebagai contoh:
- `total_rooms` tidak memperhitungkan jumlah rumah tangga
- `population` tidak mempertimbangkan ukuran distrik

Oleh karena itu, kita dapat membuat fitur rasio yang lebih representatif.

In [ ]:
housing["rooms_per_household"] = housing["total_rooms"] / housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"] / housing["total_rooms"]
housing["population_per_household"] = housing["population"] / housing["households"]

Fitur-fitur baru ini memberikan perspektif yang lebih kontekstual, misalnya:
- ukuran rumah rata-rata,
- kepadatan penduduk per rumah tangga,
- proporsi kamar tidur terhadap total kamar.

Fitur semacam ini sering kali lebih berkorelasi dengan harga rumah.

### 6.2 Checking Correlation with New Attributes

Setelah fitur baru dibuat, kita perlu mengevaluasi apakah fitur tersebut действительно memberikan informasi tambahan.

Salah satu cara cepat adalah dengan menghitung kembali korelasi terhadap target.

In [ ]:
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

Hasil korelasi menunjukkan bahwa beberapa fitur hasil rekayasa memiliki korelasi yang cukup kuat dengan `median_house_value`.

Hal ini menegaskan bahwa **feature engineering merupakan langkah krusial** dalam meningkatkan performa model.

## 7. Prepare the Data for Machine Learning Algorithms

Setelah eksplorasi data dan feature engineering, langkah selanjutnya adalah **mempersiapkan data agar siap digunakan oleh algoritma Machine Learning**.

Tahap ini sangat krusial karena sebagian besar algoritma Machine Learning **tidak dapat menangani data mentah secara langsung**, terutama jika terdapat nilai yang hilang atau fitur kategorikal.

### 7.1 Separating Predictors and Labels

Langkah pertama dalam data preparation adalah memisahkan fitur (*predictors*) dan target (*labels*).

Target dalam proyek ini adalah `median_house_value`, sehingga kolom tersebut harus dipisahkan dari fitur lainnya.

In [ ]:
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

Dengan pemisahan ini, kita memastikan bahwa proses preprocessing tidak secara tidak sengaja menggunakan informasi dari target.

### 7.2 Data Cleaning (Handling Missing Values)

Sebagaimana telah diamati sebelumnya, kolom `total_bedrooms` memiliki beberapa nilai yang hilang.

Terdapat beberapa strategi untuk menangani nilai hilang:
- menghapus baris yang mengandung nilai hilang,
- menghapus kolom yang mengandung nilai hilang,
- mengganti nilai hilang dengan nilai statistik tertentu (misalnya median).

Pada proyek ini, kita memilih pendekatan **imputasi median**.

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

Imputer hanya dapat diterapkan pada fitur numerik. Oleh karena itu, kita perlu memisahkan fitur numerik dari fitur kategorikal.

In [ ]:
housing_num = housing.drop("ocean_proximity", axis=1)

In [ ]:
imputer.fit(housing_num)

Median dari setiap fitur numerik dihitung dari training set dan akan digunakan untuk menggantikan nilai yang hilang.

In [ ]:
X = imputer.transform(housing_num)

Setelah proses imputasi, dataset numerik tidak lagi mengandung nilai yang hilang dan siap untuk tahap preprocessing berikutnya.

Untuk kemudahan analisis, hasil transformasi dapat dikembalikan ke dalam bentuk DataFrame.

In [ ]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)
housing_tr.head()

## 8. Handling Text and Categorical Attributes

Sebagian besar algoritma Machine Learning hanya dapat bekerja dengan **fitur numerik**. Oleh karena itu, fitur kategorikal perlu dikonversi ke representasi numerik sebelum digunakan untuk training.

Dalam dataset California Housing, fitur `ocean_proximity` merupakan fitur kategorikal yang menunjukkan kedekatan lokasi dengan laut.

### 8.1 Encoding Categorical Attributes

Pendekatan yang paling umum untuk menangani fitur kategorikal adalah **One-Hot Encoding**.

One-Hot Encoding mengubah setiap kategori menjadi sebuah fitur biner (0 atau 1). Dengan cara ini, model tidak mengasumsikan adanya hubungan ordinal antar kategori.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

housing_cat = housing[["ocean_proximity"]]

In [ ]:
cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

Hasil dari One-Hot Encoding berupa **sparse matrix**, yang menyimpan data secara efisien karena sebagian besar nilainya adalah nol.

Setiap kolom baru merepresentasikan satu kategori unik dari fitur `ocean_proximity`.

Untuk mengetahui kategori apa saja yang telah di-encode, kita dapat memeriksa atribut `categories_`.

In [ ]:
cat_encoder.categories_

Urutan kategori ini menentukan urutan kolom pada hasil One-Hot Encoding.

Pada praktiknya, One-Hot Encoding memastikan bahwa informasi kategorikal dapat dimanfaatkan oleh algoritma Machine Learning tanpa memperkenalkan bias numerik.

## 11. Full Pipeline for Data Preparation

Setelah membuat pipeline terpisah untuk fitur numerik, langkah selanjutnya adalah **menggabungkan preprocessing numerik dan kategorikal** ke dalam satu pipeline utuh.

Scikit-Learn menyediakan class `ColumnTransformer` untuk menerapkan transformer yang berbeda pada subset kolom yang berbeda.

### 11.1 ColumnTransformer

`ColumnTransformer` memungkinkan kita:
- menerapkan pipeline numerik pada kolom numerik,
- menerapkan One-Hot Encoding pada kolom kategorikal,
- menggabungkan seluruh hasil preprocessing menjadi satu matriks fitur.

Pendekatan ini sangat penting untuk membangun sistem Machine Learning yang bersih dan mudah dipelihara.

In [ ]:
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs),
])

Pipeline di atas memastikan bahwa:
- seluruh fitur numerik diproses secara konsisten,
- fitur kategorikal diubah menjadi representasi numerik,
- hasil akhir siap langsung digunakan oleh algoritma Machine Learning.

### 11.2 Applying the Full Pipeline

Setelah pipeline lengkap dibuat, kita dapat menerapkannya pada training set untuk menghasilkan data siap training.

In [ ]:
housing_prepared = full_pipeline.fit_transform(housing)
housing_prepared

Output dari pipeline berupa matriks numerik yang telah melalui seluruh tahap preprocessing.

Mulai dari titik ini, data siap digunakan untuk melatih berbagai model Machine Learning tanpa perlu preprocessing tambahan.

## 12. Select and Train a Model

Setelah data siap sepenuhnya, kita dapat mulai **melatih model Machine Learning**.

Pendekatan yang baik adalah memulai dari model yang **sederhana** sebagai baseline, sebelum mencoba model yang lebih kompleks.

### 12.1 Training a Linear Regression Model

Sebagai baseline, kita menggunakan **Linear Regression**. Model ini sederhana, cepat dilatih, dan memberikan gambaran awal tentang performa yang dapat dicapai.

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

Model Linear Regression telah dilatih menggunakan data yang sudah dipreprocessing sepenuhnya melalui pipeline.

### 12.2 Evaluating the Model on the Training Set

Untuk mendapatkan gambaran awal performa model, kita mengevaluasi hasil prediksi pada training set.

In [ ]:
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

Nilai RMSE ini memberikan indikasi awal seberapa baik Linear Regression memodelkan data.

Jika error masih cukup besar, hal ini menandakan bahwa model mungkin **underfitting**.

### 12.3 Training a Decision Tree Regressor

Untuk membandingkan performa, kita melatih model yang lebih kompleks, yaitu **Decision Tree Regressor**.

Decision Tree mampu memodelkan hubungan non-linear dan interaksi antar fitur.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(housing_prepared, housing_labels)

### 12.4 Evaluating the Decision Tree

Kita kembali menghitung RMSE pada training set untuk Decision Tree.

In [ ]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

Jika RMSE Decision Tree sangat kecil atau bahkan nol, hal ini justru menjadi indikasi kuat bahwa model **overfitting** pada training set.

## 13. Better Evaluation Using Cross-Validation

Evaluasi model menggunakan training set saja sering kali **menyesatkan**, terutama untuk model yang kompleks.

Untuk mendapatkan estimasi performa yang lebih realistis, digunakan teknik **cross-validation**, yang membagi data training menjadi beberapa subset dan melakukan evaluasi secara bergantian.

### 13.1 Cross-Validation for Decision Tree

Kita akan mengevaluasi Decision Tree menggunakan **K-Fold Cross-Validation** untuk melihat performa rata-rata model pada data yang tidak digunakan saat training.

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(
    tree_reg,
    housing_prepared,
    housing_labels,
    scoring="neg_mean_squared_error",
    cv=10
)

tree_rmse_scores = np.sqrt(-scores)
tree_rmse_scores

Cross-validation menunjukkan bahwa performa Decision Tree pada data validasi jauh lebih buruk dibandingkan performanya pada training set.

Hal ini mengonfirmasi bahwa model tersebut mengalami **overfitting**.

### 13.2 Cross-Validation for Linear Regression

Sebagai pembanding, kita juga melakukan cross-validation pada model Linear Regression.

In [ ]:
lin_scores = cross_val_score(
    lin_reg,
    housing_prepared,
    housing_labels,
    scoring="neg_mean_squared_error",
    cv=10
)

lin_rmse_scores = np.sqrt(-lin_scores)
lin_rmse_scores

Meskipun Linear Regression memiliki error yang lebih tinggi, performanya lebih stabil dan konsisten dibandingkan Decision Tree.

Cross-validation membantu kita memahami trade-off antara **bias dan variance** pada berbagai model.

## 14. Fine-Tune Your Model

Setelah mengevaluasi beberapa model, kita beralih ke model yang lebih kuat, yaitu **Random Forest Regressor**.

Random Forest merupakan model ensemble yang menggabungkan banyak Decision Tree untuk menghasilkan prediksi yang lebih stabil dan akurat.

### 14.1 Training a Random Forest Regressor

Random Forest bekerja dengan melatih banyak Decision Tree pada subset data dan fitur yang berbeda, kemudian menggabungkan hasil prediksinya.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(random_state=42)
forest_reg.fit(housing_prepared, housing_labels)

### 14.2 Evaluating the Random Forest Model

Seperti sebelumnya, kita mengevaluasi model menggunakan cross-validation untuk mendapatkan estimasi performa yang lebih andal.

In [ ]:
forest_scores = cross_val_score(
    forest_reg,
    housing_prepared,
    housing_labels,
    scoring="neg_mean_squared_error",
    cv=10
)

forest_rmse_scores = np.sqrt(-forest_scores)
forest_rmse_scores

Hasil cross-validation menunjukkan bahwa Random Forest memiliki error yang lebih rendah dibandingkan Linear Regression dan Decision Tree.

Hal ini menunjukkan bahwa ensemble learning mampu meningkatkan performa dengan mengurangi variance.

## 15. Fine-Tune Your Model (Hyperparameter Tuning)

Setelah memilih model yang menjanjikan, langkah selanjutnya adalah **menyetel hyperparameter** untuk mendapatkan performa terbaik.

Pada chapter ini, digunakan **GridSearchCV**, yaitu metode pencarian sistematis terhadap kombinasi hyperparameter menggunakan cross-validation.

### 15.1 Grid Search

Grid search mencoba seluruh kombinasi hyperparameter yang didefinisikan, kemudian memilih kombinasi dengan performa terbaik berdasarkan metrik evaluasi.

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]

In [ ]:
forest_reg = RandomForestRegressor(random_state=42)

grid_search = GridSearchCV(
    forest_reg,
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    return_train_score=True
)

grid_search.fit(housing_prepared, housing_labels)

Grid search di atas mengevaluasi berbagai kombinasi jumlah tree dan jumlah fitur yang digunakan pada setiap split.

Proses ini memang memakan waktu, tetapi sering kali menghasilkan peningkatan performa yang signifikan.

In [ ]:
grid_search.best_params_

Parameter terbaik yang ditemukan oleh GridSearchCV akan digunakan sebagai model final.

## 16. Evaluate Your System on the Test Set

Setelah seluruh proses training dan tuning selesai, langkah terakhir adalah mengevaluasi model menggunakan **test set**.

Test set digunakan **hanya sekali**, pada tahap akhir, untuk mendapatkan estimasi performa model yang paling realistis.

In [ ]:
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)

final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

Nilai RMSE pada test set memberikan estimasi akhir seberapa baik sistem Machine Learning bekerja pada data yang benar-benar baru.

Jika performa test set jauh lebih buruk dibandingkan training atau validation set, hal ini mengindikasikan kemungkinan overfitting.

## 17. Feature Importance

Salah satu keunggulan Random Forest adalah kemampuannya untuk memberikan estimasi **feature importance**.

Feature importance menunjukkan seberapa besar kontribusi setiap fitur terhadap prediksi model.

In [ ]:
feature_importances = final_model.feature_importances_

feature_importances

Nilai feature importance ini dapat digunakan untuk:
- memahami perilaku model,
- mengidentifikasi fitur yang paling berpengaruh,
- serta membantu proses feature selection di masa depan.

## Closing Summary (Chapter 2)

Chapter 2 memberikan gambaran lengkap mengenai **alur kerja proyek Machine Learning dari awal hingga akhir**.

Mulai dari memahami masalah bisnis, mengumpulkan dan mengeksplorasi data, melakukan preprocessing, membangun pipeline, melatih dan mengevaluasi model, hingga melakukan hyperparameter tuning dan evaluasi akhir.

Melalui chapter ini, kita belajar bahwa keberhasilan Machine Learning tidak hanya bergantung pada algoritma, tetapi juga pada kualitas data, proses persiapan, dan evaluasi yang sistematis.

Pemahaman end-to-end workflow ini menjadi fondasi yang sangat penting untuk mempelajari model dan sistem Machine Learning yang lebih kompleks pada chapter berikutnya.